Bước 1: Tiền xử lý câu văn bản

Bước 2: Word segment câu văn bản trước khi đưa vào PhoBert (do PhoBert yêu cầu)

Bước 3: Tokenize bằng bộ Tokenizer của PhoBert. Chú ý rằng khi tokenize ta sẽ thêm 2 token đặc biệt là [CLS] và [SEP] vào đầu và cuối câu.

Bước 4: Đưa câu văn bản đã được tokenize vào model kèm theo attention mask.

Bước 5: Lấy output đầu ra và lấy vector output đầu tiên (chính là ở vị trí token đặc biệt [CLS]) làm đặc trưng cho câu để train hoặc để predict (tuỳ phase)

## 1.Download library

In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdrive')
# path = "/content/gdrive/My Drive/Colab Notebooks/BERT"
# os.chdir(path)
# !ls

Mounted at /content/gdrive


In [ ]:
!pip3 install fairseq
!pip3 install fastbpe
!pip3 install vncorenlp
!pip3 install transformers
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz #download các model pretrain từ PhoBERT.
!tar -xzvf PhoBERT_base_fairseq.tar.gz # kiểm tra bên trong folder PhoBERT
!ls PhoBERT_base_fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 5.2 MB/s 
     |████████████████████████████████| 123 kB 71.1 MB/s 
     |████████████████████████████████| 241 kB 79.8 MB/s 
     |████████████████████████████████| 118 kB 81.4 MB/s 
     |████████████████████████████████| 112 kB 78.9 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141231 sha256=39ed9caa45dfd3b0d4e09ce36a464a13426de9a589115485a8c3086a2754e529
  Stored in directory: /root/.cache/pip/wheels/c8/d0/ab/d43c02eaddc5b9004db86950802442ad9a26f279c619e28da0
Successfully built antlr4-python3-runtime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fastbpe: filename=fastBPE-0.1.0-cp38-cp38-linux_x86_64.whl size=492611 sha256=6377bd790ab14a2e868a6c2d54ffdb3af1509006fe58dcd3e0aca4cda407a79f
  Stored in directo

In [ ]:
#Down load package VnCoreNLP để tokenize các câu văn.
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2022-12-27 01:48:58--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   166MB/s    in 0.2s    

2022-12-27 01:48:58 (166 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2022-12-27 01:48:59--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting respon

## 2.Tokenize Input và output


* Chuẩn bị X input: Tokenize nội dung các văn bản sang chuỗi indices.

* Chuẩn bị y output: Encoding các label output thành indices đánh dấu số thứ tự của văn bản.



In [ ]:
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/Kì 4/Deep Learning/Coursework_final/data_clean.csv", sep= "\t", index_col = None)
data.drop("Unnamed: 0", axis = 1,inplace = True)
data = data[data["sample"].isnull()==False]
data.reset_index(drop=True, inplace=True)
data

,sample,label_0
0,miếng dán bàn phím không dây logitech,Điện tử - Điện máy
1,nắp tròn gắn bàn phím bluetooth cho điện thoại...,Điện tử - Điện máy
2,pin laptop dell latitude,Điện tử - Điện máy
3,pin hp envy envy envy envy,Điện tử - Điện máy
4,đầu nối chuyển đổi dc cho laptop,Điện tử - Điện máy
...,...,...
823758,váy body lưới dài đen m,Thời trang
823759,đầm hoa nổi tay bồng cổ u s,Thời trang
823760,bộ dior chữ thêu trước ngực trắng l,Thời trang
823761,dép ny sọc đen trắng,Thời trang


In [ ]:
text_train = list(data["sample"])
label_train = list(data["label_0"])

In [ ]:
text_train

['miếng dán bàn phím không dây logitech ',
 'nắp tròn gắn bàn phím bluetooth cho điện thoại ios android',
 'pin laptop dell latitude ',
 'pin hp envy envy envy envy ',
 'đầu nối chuyển đổi dc cho laptop',
 'sạc laptop dell chân kim nhỏ chính hãng',
 'bản lề ly hợp cho macbook pro ốp',
 'hàng có sẵn đệm sưởi ấm lót chuột máy tính để bàn',
 'miếng lót chuột máy tính thay thế diy',
 'cáp chuyển đổi âm thanh usb c sang trong ',
 ' game wireless mouse optical computer mouse ergonomic mice for laptop',
 'bàn phím số với phím không dây cổng usb ',
 'máy tính xách tay chính hãng dell inspiron bàn phím',
 'bàn phím máy tính stk usb m tech',
 'vỏ bọc phím cơ pbt cho bàn phím cơ ',
 'instead mobile phone tablet stand adjustable height aluminum holder for ipad tablets',
 'removable bluetooth compatible keyboard with case for samsung galaxy tab sm ',
 ' đầu nối dây cáp cách điện ',
 'bộ chia cổng usb cổng cho máy in chuột bàn phím',
 'chuột chơi game imice có dây màu đen',
 'hộp đựng ổ cứng di động

In [ ]:
from fairseq.data.encoders.fastbpe import fastBPE  
from fairseq import options  
import numpy as np

# Khởi tạo Byte Pair Encoding cho PhoBERT

from fairseq.models.roberta import RobertaModel
phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
class BPE():
  bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'
args = BPE()
phoBERT.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

1042301B [00:00, 3069153.82B/s]
456318B [00:00, 1932718.57B/s]


In [ ]:
from tqdm import tqdm
import torch
from fairseq.data import Dictionary

max_sequence_length = 100
def convert_lines(lines, vocab, bpe):
  '''
  lines: list các văn bản input
  vocab: từ điển dùng để encoding subwords
  bpe: 
  '''
  # Khởi tạo ma trận output
  outputs = np.zeros((len(lines), max_sequence_length), dtype=np.int32) # --> shape (number_lines, max_seq_len)
  # Index của các token cls (đầu câu), eos (cuối câu), padding (padding token)
  cls_id = 0
  eos_id = 2
  pad_id = 1

  for idx, row in tqdm(enumerate(lines), total=len(lines)): 
    # Mã hóa subwords theo byte pair encoding(bpe)
    subwords = bpe.encode('<s> '+ row +' </s>')
    input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
    # Truncate input nếu độ dài vượt quá max_seq_len
    if len(input_ids) > max_sequence_length: 
      input_ids = input_ids[:max_sequence_length] 
      input_ids[-1] = eos_id
    else:
      # Padding nếu độ dài câu chưa bằng max_seq_len
      input_ids = input_ids + [pad_id, ]*(max_sequence_length - len(input_ids))
    
    outputs[idx,:] = np.array(input_ids)
  return outputs

# Load the dictionary  
vocab = Dictionary()
vocab.add_from_file("/content/PhoBERT_base_fairseq/dict.txt")


# Test encode lines
lines = ['Học_sinh được nghỉ học bắt dầu từ tháng 3 để tránh dịch covid-19', 'số lượng ca nhiễm bệnh đã giảm bắt đầu từ tháng 5 nhờ biện pháp mạnh tay']
[x1, x2] = convert_lines(lines, vocab, phoBERT.bpe)
print('x1 tensor encode: {}, shape: {}'.format(x1[:10], x1.size))
print('x1 tensor decode: ', phoBERT.decode(torch.tensor(x1))[:103])

100%|██████████| 2/2 [00:00<00:00, 623.87it/s]

x1 tensor encode: [63117  1302   884  5958    11   915   222   537   933    39], shape: 100
x1 tensor decode:  <s> Học_sinh được nghỉ học bắt dầu từ tháng 3 để tránh dịch covid-19 </s> <pad> <pad> <pad> <pad> <pad>


In [ ]:
X = convert_lines(text_train, vocab, phoBERT.bpe)
print('X shape: ', X.shape)

100%|██████████| 823763/823763 [01:54<00:00, 7184.28it/s] 

X shape:  (823763, 100)


Sau cùng ta thu được các chuỗi index có kích thước là 100, bằng với kích thước của các câu sau khi đã padding. Tiếp theo ta tạo output `y` bằng index cho các nhãn của câu.

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(label_train)
y = lb.fit_transform(label_train)
print(lb.classes_)
print('Top 4 classes indices: ', y[:4])

['Mẹ & Bé' 'Mỹ phẩm' 'Thời trang' 'Điện tử - Điện máy']
Top 4 classes indices:  [3 3 3 3]


Lưu lại dữ liệu $\mathbf{X}$ và $\mathbf{y}$

In [ ]:
# Save dữ liệu
import pickle
path = "/content/gdrive/MyDrive/Kì 4/Deep Learning/Coursework_final"
os.chdir(path)

def _save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)

def _load_pkl(path):
  with open(path, 'rb') as f:
    obj = pickle.load(f)
  return obj

## Lưu lại các files
# _save_pkl('text_train.pkl', text_train)
# _save_pkl('label_train.pkl', label_train)

# _save_pkl('X1.pkl', X)
# _save_pkl('y1.pkl', y)
# _save_pkl('labelEncoder1.pkl', lb)

# Load lại dữ liệu
X = _load_pkl('X1.pkl')
y = _load_pkl('y1.pkl')

print('length of X: ', len(X))
print('length of y: ', len(y))

length of X:  823763
length of y:  823763


## 3.Classify label by phoBERT

In [ ]:
# Load the model in fairseq
from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary

phoBERT_cls = RobertaModel.from_pretrained('/content/PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune

# Load BPE
class BPE():
  bpe_codes = '/content/PhoBERT_base_fairseq/bpe.codes'

args = BPE()
phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

# Add header cho classification với số lượng classes = 10
phoBERT_cls.register_classification_head('new_task', num_classes= 4)

tensor([[-1.3846, -1.2832, -1.7012, -1.2381]], grad_fn=<LogSoftmaxBackward0>)

In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

#Xây dựng hàm đánh giá mô hình theo 2 metric là `accuracy` và `f1_score`
def evaluate(logits, targets):
    """
    Đánh giá model sử dụng accuracy và f1 scores.
    Args:
        logits (B,C): torch.LongTensor. giá trị predicted logit cho class output.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        acc (float): the accuracy score
        f1 (float): the f1 score
    """
    # Tính accuracy score và f1_score
    logits = logits.detach().cpu().numpy()    
    y_pred = np.argmax(logits, axis = 1)
    targets = targets.detach().cpu().numpy()
    f1 = f1_score(targets, y_pred, average='weighted')
    acc = accuracy_score(targets, y_pred)
    return acc, f1

#test
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logits = torch.tensor([[0.1, 0.2, 0.7],
                       [0.4, 0.1, 0.5],
                       [0.1, 0.2, 0.7]]).to(device)
targets = torch.tensor([1, 2, 2]).to(device)
evaluate(logits, targets)

(0.6666666666666666, 0.5333333333333333)

In [ ]:
def validate(valid_loader, model, device):
    model.eval()
    accs = []
    f1s = []
    with torch.no_grad():
        for x_batch, y_batch in valid_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model.predict('new_task', x_batch)
            logits = torch.exp(outputs)
            acc, f1 = evaluate(logits, y_batch)
            accs.append(acc)
            f1s.append(f1)
    
    mean_acc = np.mean(accs)
    mean_f1 = np.mean(f1s)
    return mean_acc, mean_f1

In [ ]:
def trainOnEpoch(train_loader, model, optimizer, epoch, num_epochs, criteria, device, log_aggr = 100):
    model.train()
    sum_epoch_loss = 0
    sum_acc = 0
    sum_f1 = 0
    start = time.time()
    for i, (x_batch, y_batch) in enumerate(train_loader):
      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)
      optimizer.zero_grad()
      y_pred = model.predict('new_task', x_batch)
      logits = torch.exp(y_pred)
      acc, f1 = evaluate(logits, y_batch)
      loss = criteria(y_pred, y_batch)
      loss.backward()
      optimizer.step()

      loss_val = loss.item()
      sum_epoch_loss += loss_val
      sum_acc += acc
      sum_f1 += f1
      iter_num = epoch * len(train_loader) + i + 1

      if i % log_aggr == 0:
            print('[TRAIN] epoch %d/%d  observation %d/%d batch loss: %.4f (avg %.4f),  avg acc: %.4f, avg f1: %.4f, (%.2f im/s)'
                % (epoch + 1, num_epochs, i, len(train_loader), loss_val, sum_epoch_loss / (i + 1),  sum_acc/(i+1), sum_f1/(i+1),
                  len(x_batch) / (time.time() - start)))
      start = time.time()

In [ ]:
import os
import time
import random
import argparse
import pickle
import numpy as np
from tqdm import tqdm
from os.path import join

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn
from sklearn.model_selection import StratifiedKFold

# Load the model in fairseq
from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from transformers.modeling_utils import * 
from transformers import *

# Khởi tạo argument
EPOCHS = 20
BATCH_SIZE = 6
ACCUMULATION_STEPS = 5
FOLD = 3
LR = 0.0001
LR_DC_STEP = 80 
LR_DC = 0.1
CUR_DIR = os.path.dirname(os.getcwd())
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FOLD = 4
CKPT_PATH2 = 'model_ckpt2'

if not os.path.exists(CKPT_PATH2):
    os.mkdir(CKPT_PATH2)

# Khởi tạo DataLoader
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(X, y))

for fold, (train_idx, val_idx) in enumerate(splits):
    best_score = 0
    if fold != FOLD:
        continue
    print("Training for fold {}".format(fold))
    
    # Create dataset
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X[train_idx],dtype=torch.long), torch.tensor(y[train_idx],dtype=torch.long))
    valid_dataset = torch.utils.data.TensorDataset(torch.tensor(X[val_idx],dtype=torch.long), torch.tensor(y[val_idx],dtype=torch.long))

    # Create DataLoader
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Khởi tạo model:
    MODEL_LAST_CKPT = os.path.join(CKPT_PATH2, 'latest_checkpoint.pth.tar')
    if os.path.exists(MODEL_LAST_CKPT):
      print('Load checkpoint model!')
      phoBERT_cls = torch.load(MODEL_LAST_CKPT)
    else:
      print('Load model pretrained!')
      # Load the model in fairseq
      from fairseq.models.roberta import RobertaModel
      from fairseq.data.encoders.fastbpe import fastBPE
      from fairseq.data import Dictionary

      phoBERT_cls = RobertaModel.from_pretrained('/content/PhoBERT_base_fairseq', checkpoint_file='model.pt')
      phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune

      # # Load BPE
      # class BPE():
      #   bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

      # args = BPE()
      # phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

      # Add header cho classification với số lượng classes = 10
      phoBERT_cls.register_classification_head('new_task', num_classes=10)
      
    ## Load BPE
    print('Load BPE')
    class BPE():
      bpe_codes = '/content/PhoBERT_base_fairseq/bpe.codes'

    args = BPE()
    phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT
    phoBERT_cls.to(DEVICE)

    # Khởi tạo optimizer và scheduler, criteria
    print('Init Optimizer, scheduler, criteria')
    param_optimizer = list(phoBERT_cls.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    num_train_optimization_steps = int(EPOCHS*len(train_dataset)/BATCH_SIZE/ACCUMULATION_STEPS)
    optimizer = AdamW(optimizer_grouped_parameters, lr=LR, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)  # scheduler với linear warmup
    scheduler0 = get_constant_schedule(optimizer)  # scheduler với hằng số
    # optimizer = optim.Adam(phoBERT_cls.parameters(), LR)
    criteria = nn.NLLLoss()
    # scheduler = StepLR(optimizer, step_size = LR_DC_STEP, gamma = LR_DC)
    avg_loss = 0.
    avg_accuracy = 0.
    frozen = True
    for epoch in tqdm(range(EPOCHS)):
        # warm up tại epoch đầu tiên, sau epoch đầu sẽ phá băng các layers
        if epoch > 0 and frozen:
            for child in phoBERT_cls.children():
                for param in child.parameters():
                    param.requires_grad = True
            frozen = False
            del scheduler0
            torch.cuda.empty_cache()
        # Train model on EPOCH
        print('Epoch: ', epoch)
        trainOnEpoch(train_loader=train_loader, model=phoBERT_cls, optimizer=optimizer, epoch=epoch, num_epochs=EPOCHS, criteria=criteria, device=DEVICE, log_aggr=100)
        # scheduler.step(epoch = epoch)
        # Phá băng layers sau epoch đầu tiên
        if not frozen:
            scheduler.step()
        else:
            scheduler0.step()
        optimizer.zero_grad()
        # Validate on validation set
        acc, f1 = validate(valid_loader, phoBERT_cls, device=DEVICE)
        print('Epoch {} validation: acc: {:.4f}, f1: {:.4f} \n'.format(epoch, acc, f1))

        # Store best model checkpoint
        ckpt_dict = {
            'epoch': epoch + 1,
            'state_dict': phoBERT_cls.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        # Save model checkpoint into 'latest_checkpoint.pth.tar'
        torch.save(ckpt_dict, MODEL_LAST_CKPT)

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


Training for fold 4
Load model pretrained!


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Load BPE
Init Optimizer, scheduler, criteria


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:  0
[TRAIN] epoch 1/3  observation 0/109836 batch loss: 2.3008 (avg 2.3008),  avg acc: 0.0000, avg f1: 0.0000, (0.89 im/s)
[TRAIN] epoch 1/3  observation 100/109836 batch loss: 1.7103 (avg 1.5328),  avg acc: 0.2690, avg f1: 0.1526, (0.88 im/s)
[TRAIN] epoch 1/3  observation 200/109836 batch loss: 1.4192 (avg 1.4730),  avg acc: 0.2803, avg f1: 0.1617, (0.85 im/s)
[TRAIN] epoch 1/3  observation 300/109836 batch loss: 1.5020 (avg 1.4595),  avg acc: 0.2708, avg f1: 0.1536, (0.78 im/s)
[TRAIN] epoch 1/3  observation 400/109836 batch loss: 1.3273 (avg 1.4419),  avg acc: 0.2756, avg f1: 0.1599, (0.85 im/s)
[TRAIN] epoch 1/3  observation 500/109836 batch loss: 1.4401 (avg 1.4346),  avg acc: 0.2758, avg f1: 0.1604, (0.85 im/s)
[TRAIN] epoch 1/3  observation 600/109836 batch loss: 1.3012 (avg 1.4294),  avg acc: 0.2787, avg f1: 0.1622, (0.86 im/s)
[TRAIN] epoch 1/3  observation 700/109836 batch loss: 1.3726 (avg 1.4241),  avg acc: 0.2784, avg f1: 0.1604, (0.86 im/s)
[TRAIN] epoch 1/3  obser

Thời gian huấn luyện sẽ khá lâu, các bạn nên kiên nhẫn chờ đợi. Ngoài cách fine tuning model từ fairseq như trên, các bạn có thể tham khảo thêm một cách khác của [PhoBERT-Sentiment-Classification Khoi Nguyen](https://github.com/suicao/PhoBert-Sentiment-Classification/) thực hiện fine tuning dựa trên pretrain model huấn luyện từ transformers.